In [1]:
import pandas as pd
import dotenv
import os
dotenv.load_dotenv()

MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])



In [2]:
from nautilus_trader.data.engine import ParquetDataCatalog

print(CATALOG_PATH)
catalog = ParquetDataCatalog(CATALOG_PATH)


C:\Users\puche\code\nautilus\catalog


In [3]:
import data_utils

In [5]:
from nautilus_trader.model.identifiers import Venue, InstrumentId
from nautilus_trader.config import BacktestVenueConfig
from nautilus_trader.persistence.loaders import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.model.data import BarType, Bar
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.instruments import Instrument


start = dt_to_unix_nanos(pd.Timestamp("2022-01-01", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2023-12-02", tz="UTC"))

bar_type:BarType = BarType.from_str("EURUSD.SIM_EIGHTCAP-1-MINUTE-LAST-EXTERNAL")
instrument_id:InstrumentId = bar_type.instrument_id


In [6]:
instrument:Instrument = catalog.instruments(instrument_ids=[instrument_id.value])[0]

In [7]:
instrument

CurrencyPair(id=EURUSD.SIM_EIGHTCAP, raw_symbol=EURUSD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)

In [8]:
bar_type

BarType(EURUSD.SIM_EIGHTCAP-1-MINUTE-LAST-EXTERNAL)

In [12]:
venue_configs = [
    BacktestVenueConfig(
        name="SIM_EIGHTCAP",
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10000 USD"],
    ),
]

from nautilus_trader.config import BacktestDataConfig

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=Bar,
        instrument_id=instrument_id.value,
        start_time=start,
        end_time=end,
    ),
]
data_configs


from nautilus_trader.trading.strategy import ImportableStrategyConfig

from decimal import Decimal

strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config=dict(
            instrument_id=instrument.id.value,
            bar_type=data_utils.bar_type_to_str(bar_type),
            fast_ema_period=10,
            slow_ema_period=20,
            trade_size=Decimal(1000),
        ),
    ),
]

from nautilus_trader.config import BacktestRunConfig, BacktestEngineConfig

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)]

from nautilus_trader.backtest.node import BacktestNode

node = BacktestNode(configs)

In [13]:
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.examples.strategies.ema_cross import EMACross

result:BacktestResult = node.run()[0]

In [15]:
result

BacktestResult(trader_id='BACKTESTER-001', machine_id='DESKTOP-3FSERNM', run_config_id='218d95ea74166cd58f4baa4f8c42ac09b1ae60d832a7d686d074498c8d732812', instance_id='1d87490d-babf-4c96-8ff5-14c46e9bcb66', run_id='daf8614b-36a5-4cc8-a0b7-56f995b0ae06', run_started=1701479831920053100, run_finished=1701479852165120000, backtest_start=1693224300000000000, backtest_end=1701471600000000000, elapsed_time=8247300.0, iterations=0, total_events=26658, total_orders=8886, total_positions=4443, stats_pnls={'USD': {'PnL (total)': -320.24, 'PnL% (total)': -3.2023999999999977, 'Max Winner': 9.78, 'Avg Winner': 0.5758041329739443, 'Min Winner': 0.01, 'Min Loser': 0.0, 'Avg Loser': -0.28862162162162164, 'Max Loser': -2.84, 'Expectancy': -0.07207742516317808, 'Win Rate': 0.25050641458474004}}, stats_returns={'Returns Volatility (252 days)': 0.07191755462305727, 'Average (Return)': -2.9998776274827455e-05, 'Average Loss (Return)': -0.00024305786811782781, 'Average Win (Return)': 0.0005166704987027584, 

In [84]:
from nautilus_trader.backtest.engine import Trader

trader: Trader = node.get_engines()[0].trader
strat: EMACross = trader.strategies()[0]


In [85]:
res = node.get_engines()[0].get_result()

In [86]:
from nautilus_trader.accounting.accounts.base import Account
from nautilus_trader.portfolio import Portfolio

portfolio:Portfolio =node.get_engines()[0].portfolio

In [87]:
catalog.bars()

[Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.00730,1.02780,1.00540,1.02460,6642,946857600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02430,1.03400,1.02130,1.02920,7339,946944000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02930,1.04020,1.02840,1.03260,6570,947030400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03250,1.04150,1.02720,1.03300,7223,947116800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03290,1.03320,1.02600,1.02980,5689,947203200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02860,1.03050,1.02110,1.02510,6197,947462400000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02530,1.03410,1.02460,1.03360,6637,947548800000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03370,1.03700,1.02610,1.03080,6877,947635200000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.03090,1.03160,1.02370,1.02600,6176,947721600000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.02610,1.02750,1.01140,1.01220,6176,947808000000000000),
 Bar(EURUSD.SIM-1-DAY-LAST-EXTERNAL,1.01290,1.01480,1.00690,

In [88]:
portfolio :Portfolio = strat.portfolio

In [89]:
from nautilus_trader.portfolio.portfolio import PortfolioAnalyzer

analyzer: PortfolioAnalyzer= portfolio.analyzer

In [90]:
trader.generate_positions_report()

,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,
SIM-1-001,EMACross-000,EURUSD.SIM,SIM-001,O-20010126-0000-001-000-1,O-20010305-0000-001-000-2,SELL,FLAT,100000,2001-01-26 00:00:00+00:00,983750400000000000,2001-03-05 00:00:00+00:00,3283200000000000,0.924590187,0.9267098143,['3.69 USD'],-0.00229,-215.65 USD
SIM-1-002,EMACross-000,EURUSD.SIM,SIM-001,O-20010305-0000-001-000-3,O-20010314-0000-001-000-4,BUY,FLAT,100000,2001-03-05 00:00:00+00:00,984528000000000000,2001-03-14 00:00:00+00:00,777600000000000,0.9267098143,0.9106902383,['3.67 USD'],-0.01729,-1_605.63 USD
SIM-1-003,EMACross-000,EURUSD.SIM,SIM-001,O-20010314-0000-001-000-5,O-20010426-0000-001-000-6,SELL,FLAT,100000,2001-03-14 00:00:00+00:00,988243200000000000,2001-04-26 00:00:00+00:00,3715200000000000,0.9106902383,0.9033097812,['3.63 USD'],0.0081,734.42 USD
SIM-1-004,EMACross-000,EURUSD.SIM,SIM-001,O-20010426-0000-001-000-7,O-20010430-0000-001-000-8,BUY,FLAT,100000,2001-04-26 00:00:00+00:00,988588800000000000,2001-04-30 00:00:00+00:00,345600000000000,0.9033097812,0.8891902007,['3.59 USD'],-0.01563,-1_415.55 USD
SIM-1-005,EMACross-000,EURUSD.SIM,SIM-001,O-20010430-0000-001-000-9,O-20010717-0000-001-000-10,SELL,FLAT,100000,2001-04-30 00:00:00+00:00,995328000000000000,2001-07-17 00:00:00+00:00,6739200000000000,0.8891902007,0.8594097924,['3.50 USD'],0.03349,2_974.54 USD
SIM-1-006,EMACross-000,EURUSD.SIM,SIM-001,O-20010717-0000-001-000-11,O-20011011-0000-001-000-12,BUY,FLAT,100000,2001-07-17 00:00:00+00:00,1002758400000000000,2001-10-11 00:00:00+00:00,7430400000000000,0.8594097924,0.9024902228,['3.52 USD'],0.05013,4_304.52 USD
SIM-1-007,EMACross-000,EURUSD.SIM,SIM-001,O-20011011-0000-001-000-13,O-20011206-0000-001-000-14,SELL,FLAT,100000,2001-10-11 00:00:00+00:00,1007596800000000000,2001-12-06 00:00:00+00:00,4838400000000000,0.9024902228,0.8945097834,['3.59 USD'],0.00884,794.46 USD
SIM-1-008,EMACross-000,EURUSD.SIM,SIM-001,O-20011206-0000-001-000-15,O-20011225-0000-001-000-16,BUY,FLAT,100000,2001-12-06 00:00:00+00:00,1009238400000000000,2001-12-25 00:00:00+00:00,1641600000000000,0.8945097834,0.8790900705,['3.55 USD'],-0.01724,-1_545.52 USD
SIM-1-009,EMACross-000,EURUSD.SIM,SIM-001,O-20011225-0000-001-000-17,O-20020102-0000-001-000-18,SELL,FLAT,100000,2001-12-25 00:00:00+00:00,1009929600000000000,2002-01-02 00:00:00+00:00,691200000000000,0.8790900705,0.9033097567999999,['3.56 USD'],-0.02755,-2_425.53 USD


In [91]:
from nautilus_trader.cache.cache import Cache

c : Cache = trader._cache

In [92]:
pos = c.positions(Venue("SIM"), instrument_id)

In [1]:
from nautilus_trader.model.position import Position

positions: list[Position] = c.positions()


NameError: name 'c' is not defined

Money('-215.65', USD)